In [1]:
!wget reviews_Musical_Instruments_5.json.gz http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Musical_Instruments_5.json.gz

--2022-12-17 17:53:34--  http://reviews_musical_instruments_5.json.gz/
Resolving reviews_musical_instruments_5.json.gz (reviews_musical_instruments_5.json.gz)... failed: Name or service not known.
wget: unable to resolve host address ‘reviews_musical_instruments_5.json.gz’
--2022-12-17 17:53:34--  http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Musical_Instruments_5.json.gz
Resolving snap.stanford.edu (snap.stanford.edu)... 171.64.75.80
Connecting to snap.stanford.edu (snap.stanford.edu)|171.64.75.80|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2460495 (2.3M) [application/x-gzip]
Saving to: ‘reviews_Musical_Instruments_5.json.gz.4’

reviews_Musical_Ins 100%[===================>]   2.35M  1.32MB/s    in 1.8s    

2022-12-17 17:53:36 (1.32 MB/s) - ‘reviews_Musical_Instruments_5.json.gz.4’ saved [2460495/2460495]

FINISHED --2022-12-17 17:53:36--
Total wall clock time: 2.0s
Downloaded: 1 files, 2.3M in 1.8s (1.32 MB/s)


In [2]:
import pandas as pd
import gzip
from tqdm.notebook import tqdm
tqdm.pandas()

def parse(path):
  g = gzip.open(path, 'rb')
  for l in g:
    yield eval(l)

def getDF(path):
  i = 0
  df = {}
  for d in parse(path):
    df[i] = d
    i += 1
  return pd.DataFrame.from_dict(df, orient='index')

df = getDF('reviews_Musical_Instruments_5.json.gz')

**Идеи для извлечения упоминаний:**

1. Первое, что пришло в голову, - взять из метаданных название и искать по полному/частичному вхождению + синонимы. Если смотреть на эти данные, то часто встречается марка/модель + тип продукта (например, MXL mic), то таким образом брать первую часть или вторую и по ней искать. Или же искать по паттерну с учетом заглавных букв.

Проблемы этого подхода: 
- нет доступа к метаданным (у меня произошел connection error при скачивании, пыталась несколько раз)
- если искать по заглавным, то может попадаться много лишнего из-за наличия отзывов с капслоком для большей экспрессии


2. Была мысль извлекать синтаксические зависимости. 

Проблемы этого подхода:
- при первом приближении обнаруживается, что много коллокаций с зависимостью compound, что в целом удобно, но на одной зависимости далеко не уехать, а остальные зависимости достаточно частотные, будет попадаться много мусора


3. Очень странная идея, которая, возможно, совсем не релевантна. Можно переводить с английского языка на любой другой, тогда марки/модели товара остануться не переведенными (насколько я проверила несколько примеров в гугл переводчике). И на этой основе искать совпадения между исходным и переведенным. 

Проблемы этого подхода:
- не все указывают марку/модель, много выпадет + выпадут нарицательные сущности (гитара/микрофон)
- долго, очень долго

In [4]:
import spacy
from nltk.tokenize import wordpunct_tokenize
import nltk
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
from tqdm.notebook import tqdm
tqdm.pandas()


def clear_and_morph(text, nlp, sw): 
  text_tokens = wordpunct_tokenize(text)
  tokens_without_sw = ' '.join([word for word in text_tokens if not word in sw])
  doc = nlp(tokens_without_sw)

  return ' '.join([token.lemma_ for token in doc])

nlp = spacy.load("en_core_web_sm")
sw = stopwords.words('english')
df['reviewText_lemma'] = df['reviewText'].progress_apply(lambda x: clear_and_morph(x, nlp, sw))

/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


  0%|          | 0/10261 [00:00<?, ?it/s]

In [7]:
def clear_and_morph_nouns(text, nlp, sw): 
  text_tokens = wordpunct_tokenize(text)
  tokens_without_sw = ' '.join([word for word in text_tokens if not word in sw])
  doc = nlp(tokens_without_sw)

  return ' '.join([token.lemma_ for token in doc if token.pos_ == 'NOUN' and not token.text in sw])


sw = stopwords.words('english')
sw.extend(['product', 'year', 'device', 'use']) #в силу специфичности убираем следующие токены. да, можно для этого применить tf-idf, но от этого мы больше потеряем
df['reviewText_lemma_nouns'] = df['reviewText'].progress_apply(lambda x: clear_and_morph_nouns(x, nlp, sw))

  0%|          | 0/10261 [00:00<?, ?it/s]

In [8]:
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np


def indexing(texts):
  vectorizer = CountVectorizer(analyzer='word')
  X = vectorizer.fit_transform(texts)
  matrix_freq = np.asarray(X.sum(axis=0)).ravel()
  vect = vectorizer.get_feature_names()
  return matrix_freq, vect


def find_max(texts):
  matrix_freq, dictionary = indexing(texts)
  max_count = max(matrix_freq)
  max_ind = []
  freq_list = []
  for i in range(0, len(matrix_freq)):
      if matrix_freq[i] == max_count:
          max_ind.append(i)
  for ind in max_ind:
      freq_list.append(dictionary[ind])
  return freq_list


products = list(df['asin'].unique())
product_key = {}
for product in products:
  key = find_max(df[df['asin'] == product]['reviewText_lemma_nouns'])
  product_key[product] = key


/usr/local/lib/python3.8/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [9]:
final = [] 
for product, key in tqdm(product_key.items()):
  text_split = wordpunct_tokenize(' '.join(df[df['asin'] == product]['reviewText']))
  tokens = ' '.join(text_split)
  key_split = key[0]
  lemmas = [token.lemma_ for token in nlp(tokens)]
  for i in range(len(lemmas)):
    if lemmas[i] == key_split:
      try:
        prev = text_split[i-1]
        final.append(prev + ' ' + lemmas[i])
      except:
        pass
      try:
        next = text_split[i+2]
        final.append(lemmas[i] + ' ' + next)
      except:
        pass

  0%|          | 0/900 [00:00<?, ?it/s]

In [10]:
bigram_measures = nltk.collocations.BigramAssocMeasures()
all_text = ' '.join(df['reviewText'])
finder = nltk.collocations.BigramCollocationFinder.from_words(wordpunct_tokenize(all_text))
finder.apply_freq_filter(10)

In [11]:
student_t_result = finder.score_ngrams(bigram_measures.student_t)
chi_sq_result = finder.score_ngrams(bigram_measures.chi_sq)
pmi_result = finder.score_ngrams(bigram_measures.pmi)

In [12]:
stats = {}
for col in tqdm(final):
  col_splited = col.split()
  stats_ = {}
  for counted in student_t_result:
    if col_splited[0] == counted[0][0] and col_splited[1] == counted[0][1]:
      stats_['student_t_result'] = counted[1]
  for counted in chi_sq_result:
    if col_splited[0] == counted[0][0] and col_splited[1] == counted[0][1]:
      stats_['chi_sq_result'] = counted[1]
  for counted in pmi_result:
    if col_splited[0] == counted[0][0] and col_splited[1] == counted[0][1]:
      stats_['pmi_result'] = counted[1]
  stats[col] = stats_

  0%|          | 0/41090 [00:00<?, ?it/s]

In [13]:
df_stats = pd.DataFrame(stats).transpose()
df_stats = df_stats.dropna()

Критерий Стьюдента совсем беда

In [27]:
df_stats.sort_values(by=['student_t_result'], ascending=False)[:3]

,student_t_result,chi_sq_result,pmi_result,core
the price,29.693055,12103.731222,3.723647,price
the guitar,23.803898,2811.375821,2.251265,guitar
this pedal,23.601295,15108.805530,4.745334,pedal


Хи-квадрат и pmi неплохо справляются в контексе данный задачи, несмотря на их недостатки в целом. Если заглянуть за топ-10, то у хи-квадрата выползает больше проблем, чем у pmi. Поэтому остановимся на pmi.

In [15]:
df_stats.sort_values(by=['chi_sq_result'], ascending=False)[:5]

,student_t_result,chi_sq_result,pmi_result
pop filter,8.482730,239422.937134,11.699740
gig bag,12.195066,158116.486051,10.053137
phantom power,6.922260,55901.240639,10.187067
strap locks,10.606474,50589.131448,8.810753
build quality,10.308161,30655.124214,8.169837


In [16]:
df_stats.sort_values(by=['pmi_result'], ascending=False)[:5]

,student_t_result,chi_sq_result,pmi_result
pop filter,8.482730,239422.937134,11.699740
polishing cloth,3.462727,30214.355501,11.298861
phantom power,6.922260,55901.240639,10.187067
gig bag,12.195066,158116.486051,10.053137
boom arm,3.312708,9294.264907,9.725789


In [17]:
list_keys = list(product_key.values())
all_keys = {x for l in list_keys for x in l}

In [18]:
def get_key(pair):
  global all_keys
  lemmas = [token.lemma_ for token in nlp(pair)]
  for lemma in lemmas:
    for key in all_keys:
      if lemma == key:
        return key

In [19]:
df_stats['core'] = df_stats.index.to_series().progress_apply(lambda x: get_key(x))

  0%|          | 0/900 [00:00<?, ?it/s]

In [20]:
df_sorted = df_stats.sort_values(by=['pmi_result'], ascending=False)

In [21]:
grouped = df_sorted['core']
grouped = grouped.reset_index(level=0)
grouped = grouped[['core', 'index']]
grouped_pivot = pd.pivot_table(grouped, index=['core', 'index']) #группировка по товарам

In [22]:
#несмотря на условия дз, выводить в данном случае удобнее из исходной, там все отсортировано
print('cable')
print('---')
res = list(df_sorted[df_sorted['core'] == 'cable'].index[:5])
for item in res:
  print(item)

cable
---
Monster cable
patch cable
XLR cable
longer cable
This cable


In [23]:
print('string')
print('---')
res = list(df_sorted[df_sorted['core'] == 'string'].index[:5])
for item in res:
  print(item)

string
---
string winder
E string
12 string
G string
string changes


In [24]:
print('amp')
print('---')
res = list(df_sorted[df_sorted['core'] == 'amp'].index[:5])
for item in res:
  print(item)

amp
---
modeling amp
state amp
watt amp
practice amp
amp models


In [25]:
print('mic')
print('---')
res = list(df_sorted[df_sorted['core'] == 'mic'].index[:5])
for item in res:
  print(item)

mic
---
dynamic mic
mic stand
mic clip
this mic
mic cable


In [26]:
print('tuner')
print('---')
res = list(df_sorted[df_sorted['core'] == 'tuner'].index[:5])
for item in res:
  print(item)

tuner
---
chromatic tuner
Snark tuner
little tuner
tuner works
This tuner


**Идея по поводу бонусной части:**

Для того, чтобы объединить синонимичные упоминания, вместо лемм в данном коде нужно использовать стемы (через nltk.stem), а также с технической точки изменить способ последующего сбора пар внутри исходного текста. Сравниваем не две строки, а делаем условие на наличие стема в токене и берем этот токен.